## TODOs
1. time things
1. put summaries for the algorithms to explain what we did. This can also be taken later from the report

## Inputs

1. restaurant dimensions
1. number of timeslots?
1. marginal revenue function
1. cost of goodwill
1. randomly generated data - boolean
1. maximum table size
1. objective? - enum

## Construction Heuristic

### Algorithm

1. sort parties in order of descending size
1. add parties to the timeslot k until the restaurant capacity is full or no more tables can be added

#### Pre-work: Randomly generating a unique HEX string of 6 digits to use as a file identifier and relate logging files to data files

In [1]:
import random
import string

RG_HEX_ID = ''.join(random.choice(string.hexdigits) for _ in range(6))
print(RG_HEX_ID)

10750A


### Step 1: Parse input parameters

In [2]:
import json
import random

w = l = customer_goodwill = REVENUE_GOAL = num_timeslots = randomly_generated_data = 0
marginal_revenue = {}

# set of timeslots - static
K = 7

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    customer_goodwill = data['customer_goodwill']
    REVENUE_GOAL = data['revenue_goal']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]
        
# Restaurant squared area
MAX_SIZE = l * w

### Step 2: Randomly generate data

In [3]:
# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(8,10), 
        random.randint(10,12),
        random.randint(12,14),
        random.randint(14,16),
        random.randint(14,16),
        random.randint(14,15),
        random.randint(10,12),
    ]
    
else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of results")


#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        party_size[i, k] = random.randint(2, max_party_size)
        
#print(f"{max_party_size}\n")
#print(f"{party_size}")

### Step 3: Calculate table sizes and space that each party will occupy if seated

In [4]:
# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

### Step 4: Create and pre-populate data structures

In [5]:
sorted_space = space.copy()

sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[1], reverse=True)}
sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[0][1], reverse=False)}
#print(f"{sorted_space}")

In [6]:
# initializing a list of keys for the space dictionary
keys_list = list(sorted_space)


# looping over each of the time slot's sorted party sizes 
timeslot_sorted_parties = {}
global_counter = 0
for k in range(K):
    counter = 0
    sorted_parties=[]
    for key in sorted_space.keys():
        sorted_parties.append(sorted_space[keys_list[global_counter]])
        counter += 1
        global_counter += 1
        if(counter == I[k]):
            break
    timeslot_sorted_parties[k] = sorted_parties
print(f"{timeslot_sorted_parties}")
#print(f"{sorted_space}")
#print(f"{sorted_space[sorted_space[i],i]}")

x = [[0 for i in range(I[k])] for k in range(K)]

{0: [18, 15, 15, 15, 12, 12, 9, 9], 1: [18, 18, 15, 15, 15, 15, 12, 12, 9, 9], 2: [18, 18, 18, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12, 9], 3: [18, 18, 18, 18, 18, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 9], 4: [18, 18, 18, 18, 18, 18, 18, 15, 15, 12, 12, 12, 12, 12, 12, 9], 5: [18, 18, 18, 18, 15, 15, 15, 15, 12, 12, 12, 12, 12, 9, 9], 6: [18, 18, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12]}


### Step 5: Construction Heuristic

In [7]:
import pprint
import time

start_time = time.time()

# Observation: Sometimes we have a party size of 3/5 instead of 4/6 get allocated even though 4 will produce more revenue.
# This is because the space occupied by each is the same.
# Way we can do this is by sorting at party_size instead of space and that will propogate.
binaries = []
incumbent = []

daily_revenue = 0
counter = 0
space_used = []
for k in range(K):
    state = []
    revenue = 0
    occupied_space = 0
    #print(f"timeslot{k}")
    for i in range(I[k]):
        
        dict_to_append = {}
        size_of_party = party_size[keys_list[counter]]
        dict_to_append['party'] = i
        dict_to_append['timeslot'] = k
        dict_to_append['party_size'] = size_of_party
        
        if (occupied_space + timeslot_sorted_parties[k][i] < MAX_SIZE):
            occupied_space += timeslot_sorted_parties[k][i]
            x[k][i] = 1
            added_revenue = marginal_revenue[size_of_party] * size_of_party
            revenue += added_revenue
            #dict_to_append[i,k] = [1, added_revenue]
            binaries.append([i,size_of_party,k,1,added_revenue,revenue])
            dict_to_append['seated'] = 1
            #print(f"party size = {size_of_party}, marginal revenue = {marginal_revenue[size_of_party]}, rev={revenue}")
        else:
            dict_to_append['seated'] = 0
            binaries.append([i,size_of_party,k,'',0,revenue])
        counter+=1
        state.append(dict_to_append)
    incumbent.append(state)
    space_used.append(occupied_space)
#     print(f"{occupied_space}\n")
    daily_revenue += revenue
incumbent.append(state)
incumbent.append(daily_revenue)
incumbent.append(space_used)
# print(f"{incumbent}")

print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))



--- 0.0006539821624755859 seconds ---




### Step 6: Export results to CSV

In [8]:
# for i in binaries:
#     print(f"{i}")

#for key in sorted_space.keys():
    #print(f"{key}")
 #   print(f"{x[key[0]][key[1]]}")

In [9]:
import csv
import string

file_name = test = "construction_heuristic_" + RG_HEX_ID + ".csv"
with open(file_name, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["party i", "size of Party i", "timeslot k", "seated (binary)", "added revenue", "cum. revenue"])
    for i in binaries:
       writer.writerow(i)

In [10]:
print(f"Total Revenue: ${incumbent[8]}")
print(f"Average Space Used per Timeslot: {sum(incumbent[9]) / 7} m^2")

total_guests = 0
seated_guests = 0
total_groups = 0
seated_groups = 0
party_size_histogram = {
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}
for i in range(K):
    for j, val in enumerate(incumbent[i]):
        party_size = val['party_size']
        party_size_histogram[party_size] += 1
        total_guests += party_size
        total_groups += 1
        if (val['seated'] == 1):
            seated_guests += party_size
            seated_groups += 1
for key, value in party_size_histogram.items():
    print(f"Party Size: {key}, occurrences: {value}")
    
print(f"Average meal cost: ${incumbent[8] / seated_groups}")
print(f"Percentage of groups seated: {seated_groups / total_groups * 100}")

Total Revenue: $7393
Average Space Used per Timeslot: 97.28571428571429 m^2
Party Size: 2, occurrences: 9
Party Size: 3, occurrences: 12
Party Size: 4, occurrences: 19
Party Size: 5, occurrences: 13
Party Size: 6, occurrences: 14
Party Size: 7, occurrences: 12
Party Size: 8, occurrences: 12
Average meal cost: $171.93023255813952
Percentage of groups seated: 47.25274725274725


#### Visualizing incumbent data structure

In [11]:
# pprint.pprint(f"{incumbent}")

In [12]:
# Helper function

def calc_delta_space(removed_size, added_size):
    removed_tables = table_mapping[removed_size]
    added_tables = table_mapping[added_size]
    
    return 3 * (added_tables - removed_tables)

def calc_delta_space2(added_size):
    added_tables = table_mapping[added_size]
    
    return 3 * added_tables


### Step 7: Run Improvement Heuristic

In [13]:
import math
import time

start_time = time.time()
# 0. Order in terms of decreasing table size
# 1. Loop through all timeslots
# 2. while i < soln set
    # 2.1 take out i and replace w first zero that fits in the restaurant
    # 2.2 If it improves the obj fn then accept move and switch them out
    # 2.3 If it improves the objective function then accept move, if not reject
    
# 2. randomly select tables to add/remove within the time slot

# TODO pass all these thru the json file
og_revenue = incumbent[8]

incumbent_history = []
incumbent.append(0)
incumbent_history.append(incumbent)

challenger = incumbent.copy()
with open('log-improvement' + RG_HEX_ID + '.txt', 'w') as log_file:
    
    for i in range(K):
        which_timeslot = i
        
        first_zero = -1
        for index, entry in enumerate(incumbent[which_timeslot]):
            if entry['seated'] == 0:
                first_zero = index
                break
        
        if first_zero == -1:               
            # can't improve because all of the parties are seated
            print("do nothing")
            continue
        
        first_pointer = first_zero
        second_pointer = first_pointer + 1
        incumbent_space_used = incumbent[9][which_timeslot]
            
        
        for j, val in enumerate(incumbent[which_timeslot]):
            
            if (challenger[which_timeslot][j]['seated'] == 0):
                continue
            
            change_in_revenue = 0
            change_in_space = 0
            
            incumbent_space_used = challenger[9][which_timeslot]
            print(f"incumbent_space_used: {incumbent_space_used}")
            
            removed_party_size = challenger[which_timeslot][j]['party_size']
            removed_size = table_mapping[removed_party_size]*3 + 6
            removed_revenue = marginal_revenue[removed_party_size] * removed_party_size
            
            print(f"removed_party_size: {removed_party_size}, removed size: {removed_size}, removed revenue: {removed_revenue}")
            
            firstPotentialParty = {}
            secondPotentialParty = {}
            first_pointer_party_size = 0
            second_pointer_party_size = 0
            first_pointer_size = 0
            second_pointer_size = 0
            first_pointer_revenue = 0
            second_pointer_revenue = 0
            
            if (first_pointer < len(challenger[which_timeslot]) - 1 and challenger[which_timeslot][first_pointer]['seated']!=1):
                firstPotentialParty = challenger[which_timeslot][first_pointer]
                first_pointer_party_size = challenger[which_timeslot][first_pointer]['party_size']
                first_pointer_size = table_mapping[first_pointer_party_size]*3 + 6
                print(f"first pointer party size: {first_pointer_party_size}")
                first_pointer_revenue = first_pointer_party_size * marginal_revenue[first_pointer_party_size]
                print(f"assigned first potnetial party, party_size = {first_pointer_size}, revenue = {first_pointer_revenue}")
            
            if (second_pointer <= len(challenger[which_timeslot]) - 1 and challenger[which_timeslot][second_pointer]['seated']!=1):
                secondPotentialParty = challenger[which_timeslot][second_pointer]
                second_pointer_party_size = challenger[which_timeslot][second_pointer]['party_size']
                second_pointer_size = table_mapping[second_pointer_party_size]*3 + 6
                print(f"second pointer party size: {second_pointer_party_size}")
                second_pointer_revenue = second_pointer_party_size * marginal_revenue[second_pointer_party_size]
                print(f"assigned scond potnetial party, party_size = {second_pointer_size}, revenue = {second_pointer_revenue}")

            # greedy approach checking both first, then first, then second
            # check that size is less than total size, revenue increases
            
            #total_size = incumbent_space_used + first_pointer_size + second_pointer_size - removed_size
            total_size = challenger[9][which_timeslot] + first_pointer_size + second_pointer_size - removed_party_size
            print(f"total_size:{total_size}")
            if total_size <= MAX_SIZE and first_pointer_revenue + second_pointer_revenue > removed_revenue:
                # set revenue to new revenue, turn on first and second pointers, set space = new space
                print(f"old revenue: {challenger[8]}")
                challenger[8] += (first_pointer_revenue + second_pointer_revenue - removed_revenue)
                challenger[9][which_timeslot] += first_pointer_size + second_pointer_size - removed_party_size
                challenger[which_timeslot][j]['seated'] = 0
                challenger[which_timeslot][first_pointer]['seated'] = 1
                challenger[which_timeslot][second_pointer]['seated'] = 1 
                print(f"new revenue: {challenger[8]}")
                print(f"timeslot: {which_timeslot}, total_size: {incumbent[9][which_timeslot]}, total_rev: {first_pointer_revenue + second_pointer_revenue} both get switched on")
            elif incumbent_space_used + first_pointer_size - removed_size < MAX_SIZE and first_pointer_revenue > removed_revenue:
                challenger[8] += (first_pointer_revenue - removed_revenue)
                challenger[9][which_timeslot] += first_pointer_size - removed_party_size
                challenger[which_timeslot][j]['seated'] = 0
                challenger[which_timeslot][first_pointer]['seated'] = 1
                print(f"first gets switched on")
            elif incumbent_space_used + second_pointer_size - removed_size < MAX_SIZE and second_pointer_revenue > removed_revenue:
                challenger[8] += (second_pointer_revenue - removed_revenue)
                challenger[9][which_timeslot] += second_pointer_size - removed_party_size
                challenger[which_timeslot][j]['seated'] = 0
                challenger[which_timeslot][second_pointer]['seated'] = 1
                print(f"second gets switched on")
            else:
                print("nothing improved")
            
            print("\n\n\n\n")
            

    print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))
    print(f"original revenue: {og_revenue}, improved revenue: {challenger[8]}")
    # log_file.write("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))

incumbent_space_used: 96
removed_party_size: 8, removed size: 18, removed revenue: 240
total_size:88
nothing improved





incumbent_space_used: 96
removed_party_size: 6, removed size: 15, removed revenue: 168
total_size:90
nothing improved





incumbent_space_used: 96
removed_party_size: 6, removed size: 15, removed revenue: 168
total_size:90
nothing improved





incumbent_space_used: 96
removed_party_size: 6, removed size: 15, removed revenue: 168
total_size:90
nothing improved





incumbent_space_used: 96
removed_party_size: 3, removed size: 12, removed revenue: 69
total_size:93
nothing improved





incumbent_space_used: 96
removed_party_size: 3, removed size: 12, removed revenue: 69
total_size:93
nothing improved





incumbent_space_used: 96
removed_party_size: 2, removed size: 9, removed revenue: 52
total_size:94
nothing improved





incumbent_space_used: 96
removed_party_size: 7, removed size: 18, removed revenue: 203
first pointer party size: 4
assigned first potnetial par

### Key Stats

In [14]:
print(f"Total Revenue: ${challenger[8]}")
print(f"Average Space Used per Timeslot: {sum(challenger[9]) / 7} m^2")

total_guests = 0
seated_guests = 0
total_groups = 0
seated_groups = 0
party_size_histogram_1 = {
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}
for i in range(K):
    for j, val in enumerate(challenger[i]):
        party_size = val['party_size']
        party_size_histogram_1[party_size] += 1
        total_guests += party_size
        total_groups += 1
        if (val['seated'] == 1):
            seated_guests += party_size
            seated_groups += 1
for key, value in party_size_histogram_1.items():
    print(f"Party Size: {key}, occurrences: {value}")
    
print(f"Average meal cost: ${challenger[8] / seated_groups}")
print(f"Percentage of groups seated: {seated_groups / total_groups * 100}%")


Total Revenue: $7489
Average Space Used per Timeslot: 100.14285714285714 m^2
Party Size: 2, occurrences: 9
Party Size: 3, occurrences: 12
Party Size: 4, occurrences: 19
Party Size: 5, occurrences: 13
Party Size: 6, occurrences: 14
Party Size: 7, occurrences: 12
Party Size: 8, occurrences: 12
Average meal cost: $174.1627906976744
Percentage of groups seated: 47.25274725274725%


### 